# RNN循环神经网络
- 隐藏层之间的节点不再是无连接
- 隐藏层的输入不仅包括输入层的输出还包括上一时刻隐藏层的输出。
- 缺点：经过多次迭代容易梯度爆炸
- 实现语句：tf.nn.dynamic_rnn
## LSTM（长短时记忆 ）网络 [RNN的特殊类型]
- 拥有固定权值为1的自连接，以及一个线性激活函数，因此其局部偏导为1
- 误差可以在时间步中传递，而不会消失或爆炸
- 通过门可以让信息不通过、完全通过、部分通过
### 变体:GRU门限循环单元
- 只有一个门（更新门）：决定了内部状态与输入状态的融合比例
- 实现语句：tf.contrib.rnn.GRUCell


In [1]:
import tensorflow as tf
import numpy as np

In [5]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('C:/Users/dell/mnist/',one_hot = True)

Extracting C:/Users/dell/mnist/train-images-idx3-ubyte.gz
Extracting C:/Users/dell/mnist/train-labels-idx1-ubyte.gz
Extracting C:/Users/dell/mnist/t10k-images-idx3-ubyte.gz
Extracting C:/Users/dell/mnist/t10k-labels-idx1-ubyte.gz


In [6]:
learning_rate = 0.001
batch_size = 128
n_input = 28
n_steps = 28
n_hidden = 128
n_classes = 10

In [7]:
x = tf.placeholder(tf.float32,[None,n_steps,n_input])
y = tf.placeholder(tf.float32,[None,n_classes])

In [9]:
output,_ = tf.nn.dynamic_rnn(
            tf.contrib.rnn.GRUCell(n_hidden),
            x,
            dtype=tf.float32,
            sequence_length = batch_size*[n_input],
            )

In [10]:
output.get_shape()#batch , n_steps , n_hidden 

TensorShape([Dimension(None), Dimension(28), Dimension(128)])

In [14]:
index = tf.range(0,batch_size)*n_steps + (n_input - 1)
flat = tf.reshape(output,[-1,int(output.get_shape()[2])]) ##扁平化
last = tf.gather(flat,index)  ##提取索引处的变量

In [16]:
num_classes = int(y.get_shape()[1])
weight = tf.Variable(tf.truncated_normal([n_hidden,num_classes],stddev=0.01))
bias = tf.Variable(tf.constant(0.1,shape=[num_classes]))
prediction = tf.nn.softmax(tf.matmul(last,weight)+bias)

In [17]:
cross_entropy = -tf.reduce_sum(y*tf.log(prediction))

In [22]:
optimizer = tf.train.AdamOptimizer(learning_rate,beta1 = 0.5)
grads = optimizer.compute_gradients(cross_entropy)
for i ,(g,v) in enumerate(grads):
    if g is not None:
        grads[i] = (tf.clip_by_norm(g,5),v) # clip gradients
train_op = optimizer.apply_gradients(grads)

D:\anaconda3\lib\site-packages\tensorflow\python\ops\gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [23]:
correct_pred = tf.equal(tf.argmax(prediction,1),tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred,tf.float32))

In [24]:
sess=tf.Session()

In [25]:
init = tf.global_variables_initializer()
sess.run(init)
for step in range(1300):
    batch_x,batch_y = mnist.train.next_batch(batch_size)
    batch_x = batch_x.reshape((batch_size,n_steps,n_input))
    sess.run(train_op,feed_dict={x:batch_x,y:batch_y})
    if step%50 == 0:
        acc = sess.run(accuracy,feed_dict={x:batch_x,y:batch_y})
        loss = sess.run(cross_entropy,feed_dict={x:batch_x,y:batch_y})
        print("Iter"+str(step)+",Minibatch Loss = " + "{:.6f}".format(loss)+", Training Accuracy = " +"{:.6f}".format(acc))
print("Optimization Finished!")

Iter0,Minibatch Loss = 293.516266, Training Accuracy0.132812
Iter50,Minibatch Loss = 180.652451, Training Accuracy0.515625
Iter100,Minibatch Loss = 143.760956, Training Accuracy0.625000
Iter150,Minibatch Loss = 102.622299, Training Accuracy0.757812
Iter200,Minibatch Loss = 93.873909, Training Accuracy0.789062
Iter250,Minibatch Loss = 73.769493, Training Accuracy0.820312
Iter300,Minibatch Loss = 39.551220, Training Accuracy0.906250
Iter350,Minibatch Loss = 52.962044, Training Accuracy0.843750
Iter400,Minibatch Loss = 39.316681, Training Accuracy0.906250
Iter450,Minibatch Loss = 36.960602, Training Accuracy0.921875
Iter500,Minibatch Loss = 24.130184, Training Accuracy0.968750
Iter550,Minibatch Loss = 18.417496, Training Accuracy0.968750
Iter600,Minibatch Loss = 24.702042, Training Accuracy0.953125
Iter650,Minibatch Loss = 16.559032, Training Accuracy0.976562
Iter700,Minibatch Loss = 14.724217, Training Accuracy0.968750
Iter750,Minibatch Loss = 32.734070, Training Accuracy0.945312
Iter800

In [26]:
test_x = mnist.test.images
test_x = test_x.reshape((-1,n_steps,n_input))
test_y = mnist.test.labels

In [33]:
acc = sess.run(accuracy,feed_dict={x:test_x[:128],y:test_y[:128]})
print(acc)

1.0
